In [2]:
import pandas  as pd
import tensorflow  as tf
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import mediapipe as mp


In [3]:
import joblib
loaded_model = joblib.load('random_forest_model.pkl')

In [4]:
# List of indices for key landmarks to monitor during a deadlift
key_landmark_indices = [
    23,  # Left Hip
    24,  # Right Hip
    11,  # Left Shoulder
    12,  # Right Shoulder
    25,  # Left Knee
    26,  # Right Knee
    27,  # Left Ankle
    28,  # Right Ankle
    13,  # Left Elbow
    14,  # Right Elbow
    15,  # Left Wrist
    16   # Right Wrist
]

In [5]:
def predictions(results):
    if results.pose_landmarks:
        b=[]
        for idx,landmark in enumerate(results.pose_landmarks.landmark):
            if idx in key_landmark_indices:
                b.append(landmark.x)
                b.append(landmark.y)
                b.append(landmark.z)
        pred=loaded_model.predict(np.array(b).reshape(1,36))[0]
        prob=loaded_model.predict_proba(np.array(b).reshape(1,36))[0]
        return pred,str(max(prob))
    else:
        return "No pose detected", ""

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


In [8]:
in_rep=False
counter =0
cap = cv2.VideoCapture("ls1.mp4")
import cv2
import numpy as np

import cv2
import numpy as np

def draw_text_with_background(frame, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, font_color=(255, 255, 255), thickness=2, line_type=cv2.LINE_AA, bg_color_start=(0, 0, 255), bg_color_end=(255, 0, 0), padding=10, border_radius=5, shadow=True, alpha=0.75):
    # Calculate text size
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_x, text_y = position
    text_width, text_height = text_size

    # Calculate background rectangle coordinates
    top_left = (text_x - padding, text_y - text_height - padding)
    bottom_right = (text_x + text_width + padding, text_y + padding)

    # Create a blank image with the same dimensions as the frame
    overlay = frame.copy()

    # Draw gradient background
    if border_radius > 0:
        # Draw filled rectangle with rounded corners
        cv2.rectangle(overlay, top_left, bottom_right, bg_color_start, -1)
        cv2.circle(overlay, (top_left[0] + border_radius, top_left[1] + border_radius), border_radius, bg_color_start, -1)
        cv2.circle(overlay, (bottom_right[0] - border_radius, top_left[1] + border_radius), border_radius, bg_color_start, -1)
        cv2.circle(overlay, (top_left[0] + border_radius, bottom_right[1] - border_radius), border_radius, bg_color_start, -1)
        cv2.circle(overlay, (bottom_right[0] - border_radius, bottom_right[1] - border_radius), border_radius, bg_color_start, -1)
    else:
        cv2.rectangle(overlay, top_left, bottom_right, bg_color_start, -1)

    # Create gradient effect
    for i in range(top_left[1], bottom_right[1]):
        alpha_gradient = (i - top_left[1]) / (bottom_right[1] - top_left[1])
        color = (
            int(bg_color_start[0] * (1 - alpha_gradient) + bg_color_end[0] * alpha_gradient),
            int(bg_color_start[1] * (1 - alpha_gradient) + bg_color_end[1] * alpha_gradient),
            int(bg_color_start[2] * (1 - alpha_gradient) + bg_color_end[2] * alpha_gradient),
        )
        cv2.line(overlay, (top_left[0], i), (bottom_right[0], i), color, 1)

    # Blend overlay with the original frame
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # Draw shadow if enabled
    if shadow:
        shadow_color = (0, 0, 0)
        shadow_offset = (2, 2)
        cv2.putText(frame, text, (text_x + shadow_offset[0], text_y + shadow_offset[1]), font, font_scale, shadow_color, thickness, line_type)

    # Put text on the frame
    cv2.putText(frame, text, position, font, font_scale, font_color, thickness, line_type)


pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    # To improve performance, optionally mark the image as not writable to pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # Get predictions
    pred_text, prob_text = predictions(results)
    if pred_text=="down" and  in_rep==False :
        in_rep=True
    if pred_text=="up" and in_rep==True :
        counter=counter+1
        in_rep=False

    # Define text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)  # White color for the text
    thickness = 2
    line_type = cv2.LINE_AA
    bg_color = (0, 0, 0)  # Black color for the background rectangle

 
    # Draw text with background on the flipped image
    draw_text_with_background(image, f'Reps: {counter}', (10, 50), font, font_scale, font_color, thickness, line_type, bg_color)
    draw_text_with_background(image, pred_text, (10, 90), font, font_scale, font_color, thickness, line_type, bg_color)
    draw_text_with_background(image, prob_text, (10, 130), font, font_scale, font_color, thickness, line_type, bg_color)

    # Show the image with pose landmarks and text
    cv2.imshow('MediaPipe Pose', image)

    # Check for keypress
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Quitting...")
        break


C:\Users\godof\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\godof\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\godof\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\godof\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated.

Quitting...


C:\Users\godof\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\godof\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\godof\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\godof\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated.

# alternate text design


In [9]:
# alternate text design
import cv2
import numpy as np

def draw_text_with_background(frame, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, font_color=(255, 255, 255), thickness=2, line_type=cv2.LINE_AA, bg_color=(0, 0, 0), padding=10, border_radius=5, shadow=True):
    # Calculate text size
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_x, text_y = position
    text_width, text_height = text_size

    # Calculate background rectangle coordinates
    top_left = (text_x - padding, text_y - text_height - padding)
    bottom_right = (text_x + text_width + padding, text_y + padding)

    # Draw background rectangle with rounded corners
    if border_radius > 0:
        # Create a blank image with the same dimensions as the frame
        overlay = frame.copy()

        # Draw filled rectangle on overlay
        cv2.rectangle(overlay, top_left, bottom_right, bg_color, -1)

        # Draw rounded corners
        cv2.circle(overlay, (top_left[0] + border_radius, top_left[1] + border_radius), border_radius, bg_color, -1)
        cv2.circle(overlay, (bottom_right[0] - border_radius, top_left[1] + border_radius), border_radius, bg_color, -1)
        cv2.circle(overlay, (top_left[0] + border_radius, bottom_right[1] - border_radius), border_radius, bg_color, -1)
        cv2.circle(overlay, (bottom_right[0] - border_radius, bottom_right[1] - border_radius), border_radius, bg_color, -1)

        # Blend overlay with the original frame
        alpha = 0.75
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    else:
        cv2.rectangle(frame, top_left, bottom_right, bg_color, -1)

    # Draw shadow if enabled
    if shadow:
        shadow_color = (0, 0, 0)
        shadow_offset = (2, 2)
        cv2.putText(frame, text, (text_x + shadow_offset[0], text_y + shadow_offset[1]), font, font_scale, shadow_color, thickness, line_type)

    # Put text on the frame
    cv2.putText(frame, text, position, font, font_scale, font_color, thickness, line_type)
